# Transformed Hourly Weather Data
Author: Colin Pannikkat

This notebook transforms the Posch et. al hourly weather data into a usable input file for the GARISOM model. Soil temperature was not provided, and is instead retrieved from NLDAS in file_builder.py when building the simulation files. 

In [3]:
import pandas as pd
from datetime import datetime
import numpy as np

In [4]:
hourly_weather = pd.read_csv("./data/weather_hourly.avg_230501-231231.csv")
hourly_weather.head()

,No.,Date.Time,Rain.mm,Wind.Direction,Temp.C,RH.percent,Wind.Speed.m.s,Gust.Speed.m.s,PAR.mumol.m2.s
0,26333.5,5/1/2023 0:43,0.0,110.95,25.2760,17.60,0.00,1.260,1.2
1,26335.5,5/1/2023 1:43,0.0,57.60,23.2815,21.90,0.00,0.755,1.2
2,26337.5,5/1/2023 2:43,0.0,68.10,23.2805,23.40,0.00,1.260,1.2
3,26339.5,5/1/2023 3:43,0.0,63.20,21.7000,25.25,0.25,1.760,1.2
4,26341.5,5/1/2023 4:43,0.0,44.20,21.1990,25.55,0.50,2.010,1.2


In [5]:
new_hourly_weather = pd.DataFrame(columns=['Year', 'Day', 'Hour', 'Solar_Wm2', 'Rain_mm', 'Wind_ms.1', 'Tair_C', 'Tsoil_C', 'D_kPa'])

In [6]:
new_hourly_weather['Rain_mm'] = hourly_weather['Rain.mm']

In [7]:
new_hourly_weather['Wind_ms.1'] = hourly_weather['Wind.Speed.m.s']

In [8]:
new_hourly_weather['Tair_C'] = hourly_weather['Temp.C']

In [9]:
new_hourly_weather['Year'] = hourly_weather['Date.Time'].map(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%Y"))

In [10]:
new_hourly_weather['Day'] = hourly_weather['Date.Time'].map(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%-j"))

In [11]:
new_hourly_weather['Hour'] = hourly_weather['Date.Time'].map(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%H"))

In [12]:
def calc_e_water(T):
    '''
    Calculate saturation vapor pressure for water.
    '''
    return 6.1121 * np.exp((18.678 - (T / 234.6)) * (T / (257.14 + T)))
def calc_e_ice(T):
    '''
    Calculate saturation vapor pressure for ice.
    '''
    return 6.1115 * np.exp((23.036 - (T / 333.7)) * (T / (279.824 + T)))

In [13]:
def calc_vpd(air_temp, rh, saturation_vapor_pressure):
    '''
    Calculates VPD according to saturation vapor pressure calculations of Buck 
    (1996), these are modifications of Buck (1981) that does not require an 
    enhancement factor specification.

    VPD = e_s * (1 - RH/100)
    e_s is dependent on whether T > 0 or < 0

    air_temp must be in C, rh in percent, saturation_vapor_pressure uses Buck
    calculations which returns hPa not kPa.
    '''
    return (saturation_vapor_pressure(air_temp) * (1 - (rh / 100))) * 0.1 # 1 hPa to 0.1 kPa

In [14]:
# Calculate VPD in kPa
new_hourly_weather['D_kPa'] = hourly_weather.apply(
    lambda row: calc_vpd(row['Temp.C'], row['RH.percent'], calc_e_water) if row['Temp.C'] > 0 else calc_vpd(row['Temp.C'], row['RH.percent'], calc_e_ice),
    axis=1
)

In [15]:
def convert_par_to_solar_radiation(par):
    '''
    Conversion done per:

    Reis, Mariana & Ribeiro, Aristides. (2020). Conversion factors and general 
    quations applied in agricultural and forest meteorology. 27. 227-258. 
    10.31062/agrom.v27i2.26527. 

    "The approximation 1 W m-2 ≈ 4.57 μmol m-2 s-1 (Thimijan & Heins, 1983) is 
    assuming that the W m-2 is for photosynthetically active radiation (PAR) 
    from 4.0 to 7.0 µm."

    Sensor used for cottonwood data was HOBO S-LIA-M003, which measures
    between 400 to 700 nm, so this is fine to use, but for other sensors that do
    not measure in that range, PAR is ~2.02 instead.
    '''
    return par / 4.57

In [16]:
# Subtract weird baseline (1.2) and calculate solar radiation in Wm^-2 from micromoles/m2/s
new_hourly_weather['Solar_Wm2'] = hourly_weather['PAR.mumol.m2.s'].apply(lambda x: x - 1.2).apply(convert_par_to_solar_radiation)

In [17]:
new_hourly_weather.to_csv("./dataset.csv", index=False)